In [1]:
from zep_python import ZepClient
from zep_python import user
from pydantic import BaseModel, EmailStr
# Replace with Zep API URL and (optionally) API key
client = ZepClient( api_url)


In [ ]:
class UserRequest(BaseModel):
    user_id:str
    email:EmailStr
    first_name:str
    last_name:str
    metadata:dict

In [23]:
user_request = UserRequest(
    user_id='user1223',
    email="user@example.com",
    first_name="Jane",
    last_name="Smith",
    metadata={"foo": "bar"}
)
new_user = client.user.add(user_request)

In [2]:
import google.generativeai as genai
from src.llm.data import Prompt
from src.llm.engine import LLM
from src.llm.configs import LLM_CONFIGS
genai.configure(api_key=LLM_CONFIGS['GEMINI_API_KEY'])

c:\Users\ialrayes\AppData\Local\anaconda3\envs\SAMA\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


ModuleNotFoundError: No module named 'configs'

In [10]:
from user import get_user
get_user('user1223')

User(uuid=UUID('af98dc52-db7b-4583-bb7b-06b67e81e1f8'), id=4, created_at=datetime.datetime(2024, 6, 1, 17, 41, 41, 969325, tzinfo=datetime.timezone.utc), updated_at=datetime.datetime(2024, 6, 1, 17, 41, 41, 969325, tzinfo=datetime.timezone.utc), deleted_at=None, user_id='user1223', email='user@example.com', first_name='Jane', last_name='Smith', metadata={'foo': 'bar'})

In [ ]:
client.user.add_session

In [33]:
import asyncio
import logging
from zep_python import ZepClient, Session
from pydantic import BaseModel, EmailStr
from typing import Optional, Dict, List
import uuid

# Configure logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

API_URL = "http://localhost:8000"

class UserRequest(BaseModel):
    user_id: str
    email: EmailStr
    first_name: str
    last_name: str
    metadata: Optional[Dict] = None

class SessionRequest(BaseModel):
    user_id: str
    session_data: Dict


def get_client() -> ZepClient:
    return ZepClient(API_URL)

# User Management
async def create_user(request: UserRequest) -> None:
    async with get_client() as client:
        try:
            await client.user.aadd(request)
            logger.info("User created successfully.")
        except Exception as e:
            logger.error(f"Error creating user: {e}")

async def get_user(user_id: str) -> Optional[Dict]:
    async with get_client() as client:
        try:
            return await client.user.aget(user_id)
        except Exception as e:
            logger.error(f"Error fetching user: {e}")
            return None

async def get_all_users() -> Optional[List[Dict]]:
    async with get_client() as client:
        try:
            return await client.user.alist()
        except Exception as e:
            logger.error(f"Error fetching users: {e}")
            return None

async def update_user(request: UserRequest) -> None:
    async with get_client() as client:
        try:
            await client.user.aupdate(request)
            logger.info("User updated successfully.")
        except Exception as e:
            logger.error(f"Error updating user: {e}")

async def delete_user(user_id: str) -> None:
    async with get_client() as client:
        try:
            await client.user.adelete(user_id)
            logger.info("User deleted successfully.")
        except Exception as e:
            logger.error(f"Error deleting user: {e}")

# Session Management
async def add_session(user_id: str, metadata: dict = {}) -> None:
    async with get_client() as client:
        session_id = uuid.uuid4().hex
        session = Session(
            session_id=session_id,
            user_id=user_id,
            metadata=metadata
        )
        try:
            await client.memory.aadd_session(session)
            logger.info(f"Session added successfully with ID: {session_id}")
        except Exception as e:
            logger.error(f"Error adding session: {e}")

async def get_session(session_id: str) -> Optional[Dict]:
    async with get_client() as client:
        try:
            return await client.memory.aget_session(session_id)
        except Exception as e:
            logger.error(f"Error fetching session: {e}")
            return None

async def delete_session(session_id: str) -> Optional[Dict]:
    async with get_client() as client:
        try:
            return await client.memory.adelete_memory(session_id)
        except Exception as e:
            logger.error(f"Error deleting session: {e}")
            return None

async def get_all_session_messages(session_id: str) -> None:
    async with get_client() as client:
        try:
            messages = await client.message.aget_session_messages(session_id)
            for message in messages:
                logger.info(message.to_dict())
        except Exception as e:
            logger.error(f"Error fetching session messages: {e}")
            
            
# Memory Management
async def get_memory(session_id: str, lastin: int =5) -> Optional[Dict]:
    async with get_client() as client:
        try:
            return await client.memory.aget_memory(session_id)
        except Exception as e:
            logger.error(f"Error getting memory: {e}")
            return None
async def get_memory(session_id: str, memory_messages) -> Optional[Dict]:
    async with get_client() as client:
        try:
            return await client.memory.aadd_memory(session_id)
        except Exception as e:
            logger.error(f"Error getting memory: {e}")
            return None


In [34]:
user_request = UserRequest(
    user_id="user123",
    email="user123@example.com",
    first_name="John",
    last_name="Doe",
    metadata={"key": "value"}
)
await create_user(user_request)

# Retrieve the created user
user = await get_user("user123")
print("Retrieved User:", user)

# Update the user
user_request.first_name = "Jane"
await update_user(user_request)

# Retrieve all users
users = await get_all_users()
print("All Users:", users)

# Add a session for the user
await add_session(user_id="user123", metadata={"session_key": "session_value"})

# Retrieve the session
session = await get_session("fd16b1dee76a4b7785cc1b5b50eeff18")
print("Retrieved Session:", session)

# Delete the session
await delete_session("user123")

# Delete the user
await delete_user("user123")

INFO:httpx:HTTP Request: GET http://localhost:8000/healthz "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://localhost:8000/api/v1/user "HTTP/1.1 201 Created"
INFO:__main__:User created successfully.
INFO:httpx:HTTP Request: GET http://localhost:8000/healthz "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: GET http://localhost:8000/api/v1/user/user123 "HTTP/1.1 200 OK"


Retrieved User: uuid=UUID('e0fad46a-7ea7-463e-8317-f64445c97257') id=21 created_at=datetime.datetime(2024, 6, 1, 21, 27, 13, 263125, tzinfo=datetime.timezone.utc) updated_at=datetime.datetime(2024, 6, 1, 21, 27, 13, 263125, tzinfo=datetime.timezone.utc) deleted_at=None user_id='user123' email='user123@example.com' first_name='John' last_name='Doe' metadata={'key': 'value'}


INFO:httpx:HTTP Request: GET http://localhost:8000/healthz "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: PATCH http://localhost:8000/api/v1/user/user123 "HTTP/1.1 200 OK"
INFO:__main__:User updated successfully.
INFO:httpx:HTTP Request: GET http://localhost:8000/healthz "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: GET http://localhost:8000/api/v1/user?limit=&cursor= "HTTP/1.1 200 OK"


All Users: [User(uuid=UUID('af98dc52-db7b-4583-bb7b-06b67e81e1f8'), id=4, created_at=datetime.datetime(2024, 6, 1, 17, 41, 41, 969325, tzinfo=datetime.timezone.utc), updated_at=datetime.datetime(2024, 6, 1, 17, 41, 41, 969325, tzinfo=datetime.timezone.utc), deleted_at=None, user_id='user1223', email='user@example.com', first_name='Jane', last_name='Smith', metadata={'foo': 'bar'}), User(uuid=UUID('e0fad46a-7ea7-463e-8317-f64445c97257'), id=21, created_at=datetime.datetime(2024, 6, 1, 21, 27, 13, 263125, tzinfo=datetime.timezone.utc), updated_at=datetime.datetime(2024, 6, 1, 21, 27, 16, 588687, tzinfo=datetime.timezone.utc), deleted_at=None, user_id='user123', email='user123@example.com', first_name='Jane', last_name='Doe', metadata={'key': 'value'})]


INFO:httpx:HTTP Request: GET http://localhost:8000/healthz "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://localhost:8000/api/v1/sessions "HTTP/1.1 201 Created"
INFO:__main__:Session added successfully with ID: ad3b3b46c4ad458bb666d86b63578523
INFO:httpx:HTTP Request: GET http://localhost:8000/healthz "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: GET http://localhost:8000/api/v1/sessions/fd16b1dee76a4b7785cc1b5b50eeff18 "HTTP/1.1 404 Not Found"
ERROR:__main__:Error fetching session: No session found for session fd16b1dee76a4b7785cc1b5b50eeff18: None


Retrieved Session: None


INFO:httpx:HTTP Request: GET http://localhost:8000/healthz "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: DELETE http://localhost:8000/api/v1/sessions/user123/memory "HTTP/1.1 404 Not Found"
ERROR:__main__:Error deleting session: No query results found: None
INFO:httpx:HTTP Request: GET http://localhost:8000/healthz "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: DELETE http://localhost:8000/api/v1/user/user123 "HTTP/1.1 200 OK"
INFO:__main__:User deleted successfully.


In [36]:
client.user.list()

INFO:httpx:HTTP Request: GET http://localhost:8000/api/v1/user?limit=&cursor= "HTTP/1.1 200 OK"


[User(uuid=UUID('af98dc52-db7b-4583-bb7b-06b67e81e1f8'), id=4, created_at=datetime.datetime(2024, 6, 1, 17, 41, 41, 969325, tzinfo=datetime.timezone.utc), updated_at=datetime.datetime(2024, 6, 1, 17, 41, 41, 969325, tzinfo=datetime.timezone.utc), deleted_at=None, user_id='user1223', email='user@example.com', first_name='Jane', last_name='Smith', metadata={'foo': 'bar'})]

In [38]:
client.memory.get_memory('52e5cdcb423746e2b5f95c4f5e6abb9b')

INFO:httpx:HTTP Request: GET http://localhost:8000/api/v1/sessions/52e5cdcb423746e2b5f95c4f5e6abb9b/memory "HTTP/1.1 200 OK"


Memory(messages=[], metadata=None, summary=Summary(uuid='00000000-0000-0000-0000-000000000000', created_at='0001-01-01T00:00:00Z', content='', recent_message_uuid='00000000-0000-0000-0000-000000000000', token_count=0), uuid=None, created_at=None, token_count=None)

In [ ]:
client.memory.get_memory()

AttributeError: type object 'Memory' has no attribute 'summary'